In [1]:
from multiprocessing import Pool, cpu_count
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from gensim.models import doc2vec
from pprint import pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import numpy as np
import random
import smart_open
from collections import defaultdict

def train_vectors(path, local_context=150, name=''):
    model = doc2vec.Doc2Vec(corpus_file=path, vector_size=2, negative=5, sample=1e-5, dbow_words=1,
                               min_count=10, window=local_context, workers=4, epochs=30)
    model.train(corpus_file=path, total_words=model.corpus_count, epochs=model.epochs)
    model.save(name if name else 'trained_model{}'.format(local_context))

In [2]:
modeldm = doc2vec.Doc2Vec.load('/Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained')
modelcbow = doc2vec.Doc2Vec.load('/Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/glove-vectors-final2/model150_trained_cbow')
# modelcbow50 = doc2vec.Doc2Vec.load('../vectors/doc2vec_data/glove-vectors-final2/model50_trained_cbow')
# modeldm50 = doc2vec.Doc2Vec.load('../vectors/doc2vec_data/data50/model50_trained')
# concat_model = ConcatenatedDoc2Vec(models=[modeldm, modelcbow])

2019-02-24 21:32:43,774 : INFO : loading Doc2Vec object from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained
2019-02-24 21:32:50,369 : INFO : loading vocabulary recursively from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained.vocabulary.* with mmap=None
2019-02-24 21:32:50,370 : INFO : loading docvecs recursively from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained.docvecs.* with mmap=None
2019-02-24 21:32:50,373 : INFO : loading vectors_docs from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained.docvecs.vectors_docs.npy with mmap=None
2019-02-24 21:32:50,959 : INFO : loading wv recursively from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/model150_trained.wv.* with mmap=None
2019-02-24 21:32:50,961 : INFO : loading trainables recursively from /Users/michalkukielka/Desktop/licencjat/vectors/doc2vec_data/data150/mod

Retriece most similar documents

In [3]:
def most_similar_docs(path, model, doc_id):
    sims = model.docvecs.most_similar(doc_id, topn=10)
    ids = list(map(lambda x: x[0], sims))
    return get_docs_for_ids(path, ids)

def get_docs_for_ids(path, ids):
    if not isinstance(ids, list):
        ids = list(ids)
    docs = []
    with smart_open.smart_open(path, 'r+') as f:
        for index, tokens in enumerate(f):
            if index in ids:
                docs.append('{}: {}'.format(index , tokens))
    return docs

ICD10 codes

In [4]:
icd10 = defaultdict()
with open('../vectors/icd10cm_codes_2018.txt', 'r+') as f:
    for line in f.readlines():
        line = line.split()
        icd10[line[0]] = ' '.join(line[1:])

Get most similar words for code

In [5]:
def most_similar_for_icd10(model, codes=icd10):
    results = defaultdict()
    for key in icd10.keys():
        try:
            results[key] = modelcbow.wv.most_similar(key.lower(), topn=20)
        except KeyError:
            pass
    return results

In [ ]:
results_cbow150 = most_similar_for_icd10(model=modelcbow, codes=icd10)
results_cbow50 = most_similar_for_icd10(model=modelcbow50, codes=icd10)

Saving output

In [ ]:
with open('doc2vec_output50', 'w+') as output_file:  
    for key, words in results_cbow50.items():
        output_file.write('{}: {}\n'.format(key, ' '.join(
            list(map(lambda x: x[0], words)))))

Find most similar docs by icd10 title

In [14]:
# infer all titles in order to add new words to vocabulary
for key, title in icd10.items():
    vector = modelcbow.infer_vector(title.lower().split())
    most_similar_docs(doc_id=[x], model=modelcbow, path='../vectors/corpus.txt')